# Evaluate Bot or Not Quality

The purpose is to evaluate the bot_or_not labels accuracy with manual inspection



In [3]:
import polars as pl
import os
pl.Config.set_fmt_str_lengths(400)
pl.Config.set_tbl_rows(30)
from sklearn.metrics import confusion_matrix

polars.config.Config

In [5]:
DATA_PATH = os.getenv("DATA_PATH", "")

In [14]:
bot_or_not = pl.read_parquet(f"{DATA_PATH}/interim/bot_or_not_without_info.parquet")
bot_or_not

fid,bot
i64,bool
446097,false
3,false
8,false
12,false
2,false
17,false
18,false
20,false
23,false


In [16]:
bot_or_not["bot"].value_counts()

bot,count
bool,u32
true,4637
false,7428


In [4]:
fnames = pl.read_parquet(f"{DATA_PATH}/raw/farcaster-fnames-0-1730134800.parquet")
last_fnames = fnames[["fid","updated_at"]].group_by("fid").max()
last_fnames = last_fnames.join(fnames,on=["fid","updated_at"],how="left",coalesce=True)[["fid","fname"]]
# will be used in "3. Check bot label changes from sybilscar"
last_fnames

fid,fname
i64,str
849301,"""paulissonmiura"""
830404,"""fspoj8202"""
438114,"""digua"""
237397,"""guroux.eth"""
259801,"""xcson"""
265991,"""ellipticpoint"""
760947,"""veracious074"""
425889,"""across11"""
568757,"""snowoman"""


In [5]:
df = bot_or_not.join(last_fnames,on="fid",coalesce=True,how="left")
selected_samples = df.sample(30,seed=1)
selected_samples

fid,bot,fname
i64,bool,str
434394,true,"""spyderman1234"""
4378,false,"""gigamesh.eth"""
241627,false,"""declank"""
322865,false,"""wellborn"""
385003,false,"""manbosan"""
359645,false,"""mirrortune"""
296687,false,"""jonathancolton"""
434600,true,"""asf"""
410043,true,"""heromartin"""


In [6]:
# result_dict = {str(value): True for value in selected_samples['fname']}
# result_dict

In [7]:
manual_inspection = {
'spyderman1234': True,
 'gigamesh.eth': False,
 'declank': False,
 'wellborn': True,
 'manbosan': True,
 'mirrortune': True,
 'jonathancolton': True,
 'asf': True,
 'heromartin': True,
 'suhud15': True,
 'naser1377': True,
 'kaitooo.eth': True,
 'amangoswami': True,
 'hoangtrung': True,
 'royadelsa': True ,
 'gnarking': True ,
 'kick-your-brain': True ,
 'davoodi': True ,
 'ladysa': True ,
 'userbox': True ,
 'anhelia': False ,
 'elcamino': True ,
 'sepiata': True ,
 'suv': True ,
 'okommpeople': True ,
 'alirahmanifard': False ,
 'duongdungg': True ,
 'biterror.eth': False ,
 'kati47': True ,
 'stevedylandev.eth': False }

In [8]:
df = selected_samples.with_columns(
    pl.col("fname")
    .map_elements(lambda x: manual_inspection.get(str(x), None))
    .alias("manual_inspection")
)

df

/var/folders/k2/v9m0s0cn02j1r54h8qh_tf200000gn/T/ipykernel_53050/2440418188.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = selected_samples.with_columns(


fid,bot,fname,manual_inspection
i64,bool,str,bool
434394,true,"""spyderman1234""",true
4378,false,"""gigamesh.eth""",false
241627,false,"""declank""",false
322865,false,"""wellborn""",true
385003,false,"""manbosan""",true
359645,false,"""mirrortune""",true
296687,false,"""jonathancolton""",true
434600,true,"""asf""",true
410043,true,"""heromartin""",true


In [10]:
true_vals = df['manual_inspection'].to_numpy()
pred_vals = df['bot'].to_numpy()

# Compute confusion matrix
cm = confusion_matrix(true_vals, pred_vals)
cm

array([[ 5,  1],
       [15,  9]])

In [11]:
accuracy = (cm[0][0] + cm[1][1]) / (cm.sum().sum())
print(f'Accuracy: {100*accuracy:.2f}%')

true_bot_rate = (cm[1][1]) / (cm[1].sum())
print(f'True bot rate: {100*true_bot_rate:.2f}%')

true_human_rate = (cm[0][0]) / (cm[0].sum())
print(f'True human rate: {100*true_human_rate:.2f}%')

Accuracy: 46.67%
True bot rate: 37.50%
True human rate: 83.33%


### Conclusion

In general, bot or not seems to be pretty careful when labeling bots, with a pretty low True bot rate.

<style>

  .black {
    color:"black";
  }

  .row {
    display:flex;
    flex-direction:row;
    justify-items:center;
    align-items:center;
    gap:10px;
    margin-bottom:10px;
  }

  .column {
    display:flex;
    flex-direction:column;
    justify-items:center;
    align-items:center;
    max-width:30%;
  }

  .human {
    height:20px;
    width:20px;
    border: 2px solid #000;
    background-color: #ccc;
    border-radius:10px;
    text-align:center;
  }
  .sybil {
    height:20px;
    width:20px;
    border: 2px solid #000;
    background-color: #cc3333;
    border-radius:10px;
    text-align:center;
    color:"black";
  }

</style>

The current Bot or Not data can be approximately represented as this:

<div class="row">
  <div class="column">
    <div class="row">
      <div class="human"></div>
      <div class="human"></div>
      <div class="human"></div>
      <div class="human"></div>
      <div class="human"></div>
    </div>
    <div class="row">
      <div class="human"><span style="color:black">x</span></div>
      <div class="human"><span style="color:black">x</span></div>
      <div class="human"><span style="color:black">x</span></div>
      <div class="human"><span style="color:black">x</span></div>
      <div class="human"><span style="color:black">x</span></div>
    </div>
    <div class="row">
      <div class="human"><span style="color:black">x</span></div>
      <div class="human"><span style="color:black">x</span></div>
      <div class="human"><span style="color:black">x</span></div>
      <div class="human"><span style="color:black">x</span></div>
      <div class="human"><span style="color:black">x</span></div>
    </div>
    <div class="row">
      <div class="human"><span style="color:black">x</span></div>
      <div class="human"><span style="color:black">x</span></div>
      <div class="human"><span style="color:black">x</span></div>
      <div class="human"><span style="color:black">x</span></div>
      <div class="human"><span style="color:black">x</span></div>
    </div>
    <div class="row">
      <div class="sybil"><span style="color:black">x</span></div>
      <div class="sybil"></div>
      <div class="sybil"></div>
      <div class="sybil"></div>
      <div class="sybil"></div>
    </div>
    <div class="row">
      <div class="sybil"></div>
      <div class="sybil"></div>
      <div class="sybil"></div>
      <div class="sybil"></div>
      <div class="sybil"></div>
    </div>
  </div>
  <div class="column">
    <div class="row">
      <span style="color:white">Legend:</span>
    </div>
    <div class="row">
      <div class="human"></div>
      <span style="color:white">Bot or Not says its Human</span>
    </div>
    <div class="row">
      <div class="sybil"></div>
      <span style="color:white">Bot or Not says its Sybil</span>
    </div>
    <div class="row">
      <span style="color:white; height:20px; width:20px; text-align:center; margin: 0 2px;">X</span>
      <span style="color:white">Bot or Not is wrong</span>
    </div>
  </div>
</div>

There are two main options:

1. Remove all human labels

-  ~90% accuracy
- only bots in dataset
- 4.3k samples

2. Set everyone as sybil

- ~80% accuracy
- only bots in dataset
- 12k samples
- we would like to improve the accuracy, somehow filtering the 20% humans

In any case, we will need to get new human labels